## Assignment 2

## Submitted by: Pranav Saboo & Akshay Madar

In [32]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import time
import requests
time_counter = time.time()
import sklearn
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import os

# Question 1

In [33]:
##Read files
os.chdir('C:\\Users\\prana\\Downloads\\Web Data\\Assignment 2')
panel = pd.read_csv('panel.csv')
yelp = pd.read_csv('yelp.csv')

## cleaning data - adding "year" column and cleaning the rating column

yelp['year'] = pd.DatetimeIndex(yelp['review_date']).year
yelp['review_rating'] = yelp["review_rating"].str.split(".0", n = 1, expand = True)
yelp['review_rating'] = yelp['review_rating'].astype('int')

df_yelp = yelp.copy()
df_yelp_nycvis,df_yelp_nycres = yelp.copy(),yelp.copy()

## 1a, 1b

In [34]:
# 1a, 1b

## full yelp dataset - group by year and zip code, count review text and average review ratings & join to panel

df_yelp = df_yelp.groupby(['year', 'address_zipcode']).agg({'review_text':'count','review_rating': 'mean'}).reset_index()
df_yelp= df_yelp.rename(columns = {'review_rating': 'yelp_average', 'address_zipcode' : 'zipcode', 'review_text' : 'yelp_count'})
panel = pd.merge(panel, df_yelp, how ='left', on = ['year', 'zipcode'])

## 1c, 1d

In [35]:
# 1c, 1d

## NYC VISITORS from the yelp dataset - group by year and zip code, count review text and average review ratings

df_yelp_nycvis = df_yelp_nycvis[df_yelp_nycvis['NYC_reviewer'] == 0]
df_yelp_nycvis = df_yelp_nycvis.groupby(['year', 'address_zipcode']).agg({'review_text':'count','review_rating': 'mean'}).reset_index()
df_yelp_nycvis= df_yelp_nycvis.rename(columns = {'review_rating': 'yelp_NYC_average', 'address_zipcode' : 'zipcode', 'review_text' : 'yelp_NYC_count'})

## join to panel
panel = pd.merge(panel, df_yelp_nycvis, how ='left', on = ['year', 'zipcode'])

## NYC RESIDENTS from the yelp dataset - group by year and zip code, count review text and average review ratings

df_yelp_nycres = df_yelp_nycres[df_yelp_nycres['NYC_reviewer'] == 1]
df_yelp_nycres = df_yelp_nycres.groupby(['year', 'address_zipcode']).agg({'review_text':'count','review_rating': 'mean'}).reset_index()
df_yelp_nycres= df_yelp_nycres.rename(columns = {'review_rating': 'yelp_visitors_average', 'address_zipcode' : 'zipcode', 'review_text' : 'yelp_visitors_count'})

## join to panel
panel = pd.merge(panel, df_yelp_nycres, how ='left', on = ['year', 'zipcode'])

## 1e

In [14]:
##writing output to csv

panel.to_csv("panel_output.csv")

# Question 2

## Refer R file

# Question 3

In [18]:
#creating copies
df_yelp_res, df_yelp_vis = yelp.copy(), yelp.copy()
df_yelp_res = df_yelp_res[df_yelp_res['NYC_reviewer'] ==1]
df_yelp_vis = df_yelp_vis[df_yelp_vis['NYC_reviewer'] == 0]

In [21]:
docs_res = df_yelp_res['review_text'].to_list()
azure_phrases_res = []

s = 1
e = 1
batch = 1

for text in docs_res:
    
    if s % 999 ==0:
        print("waiting")
        print(abs(75 - (time.time() - time_counter)))
        time.sleep(75)
        time_counter = time.time()
        batch +=1
        print('Detecting batch:', batch)
    
    s += 1   
    
    # Azure portal URL.
    base_url = 'https://pranav.cognitiveservices.azure.com/'
    # The unique account key goes here.
    account_key = '3c726ae25a1f4c73bd397ed7c969a212'

    headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

    documents = {"documents":[{"id":"1","text":text}]} 

    # #Detect Sentiment
    batch_keyphrase_url = base_url + "/text/analytics/v2.1/keyPhrases"
    headers   = {'Ocp-Apim-Subscription-Key': account_key}
    response  = requests.post(batch_keyphrase_url, headers=headers, json=documents)
    phrase = response.json()
    
    try:
        azure_phrases_res.append(phrase['documents'][0]['keyPhrases'])
    except:
        print("Error block:", e)
        e+=1
        time.sleep(60)
        phrase = response.json()
        azure_phrases_res.append(phrase['documents'][0]['keyPhrases'])

waiting
691.8550155162811
Detecting batch: 2
waiting
382.84392261505127
Detecting batch: 3
waiting
393.17654609680176
Detecting batch: 4
waiting
375.60513973236084
Detecting batch: 5
waiting
380.5275504589081
Detecting batch: 6
waiting
374.2600200176239
Detecting batch: 7
waiting
382.59374475479126
Detecting batch: 8
waiting
402.3008441925049
Detecting batch: 9
waiting
389.3003420829773
Detecting batch: 10


In [23]:
df_yelp_res['phrase'] = azure_phrases_res
df_yelp_res.to_csv('resident.csv', index = False)

In [52]:
dict_nyc =dict()
for i in azure_phrases_res:
#     print(i)
    for j in i:
        j = j.lower()
        dict_nyc[j] = dict_nyc.get(j,0) + 1
df_res = pd.DataFrame.from_dict(dict_nyc, orient='index')
df_res = df_res.sort_values(by =0, ascending = False)
df_res = df_res.rename(columns ={0: 'count'})
df_res.to_csv('res_phrase.csv')

In [53]:
df_res

,count
food,3020
place,2682
service,1707
time,1300
staff,905
...,...
precious hours,1
pizza spot,1
chicken bones,1
hasta leugo,1


In [48]:
word_res =[]
for phrase in azure_phrases_res:
    for i in phrase:
        l = i.split(' ')
        for j in l:
            j = j.lower()
            word_res.append(j)

dict_word_res =dict()
for i in word_res:
    dict_word_res[i] = dict_word_res.get(i,0) + 1
df_word_res = pd.DataFrame.from_dict(dict_word_res, orient='index')
df_word_res = df_word_res.sort_values(by =0, ascending = False)
df_word_res = df_word_res.rename(columns ={0: 'count'})
df_word_res.to_csv('res_word.csv')

In [49]:
df_word_res

,count
food,5956
of,4793
place,4177
service,3336
great,2597
...,...
price2,1
practices,1
vanessa's,1
disturbing,1


In [36]:
docs_vis = df_yelp_vis['review_text'].to_list()
azure_phrases_vis = []
s = 1
e=1
batch = 1

for text in docs_vis:
    
    if s % 999 ==0:
        print("waiting")
        print(abs(75 - (time.time() - time_counter)))
        time.sleep(100)
        time_counter = time.time()
        batch +=1
        print('Detecting batch:', batch)
    
    s += 1   
    
    # Azure portal URL.
    base_url = 'https://pranav.cognitiveservices.azure.com/'
    # The unique account key goes here.
    account_key = '3c726ae25a1f4c73bd397ed7c969a212'

    headers = {'Content-Type':'application/json', 'Ocp-Apim-Subscription-Key':account_key}

    documents = {"documents":[{"id":"1","text":text}]} 

    # #Detect Sentiment
    batch_keyphrase_url = base_url + "/text/analytics/v2.1/keyPhrases"
    headers   = {'Ocp-Apim-Subscription-Key': account_key}
    response  = requests.post(batch_keyphrase_url, headers=headers, json=documents)
    phrase = response.json()
    
    try:
        azure_phrases_vis.append(phrase['documents'][0]['keyPhrases'])
    except:
        print("Error block:", e)
        e+=1
        time.sleep(60)
        phrase = response.json()
        azure_phrases_vis.append(phrase['documents'][0]['keyPhrases'])

waiting
1499.0875263214111
Detecting batch: 2
waiting
389.1264274120331
Detecting batch: 3
waiting
414.2844502925873
Detecting batch: 4
waiting
389.09541749954224
Detecting batch: 5
waiting
388.5207600593567
Detecting batch: 6
waiting
393.7106750011444
Detecting batch: 7
waiting
384.4569637775421
Detecting batch: 8
waiting
386.4087607860565
Detecting batch: 9
waiting
377.4249711036682
Detecting batch: 10


In [37]:
df_yelp_vis['phrase'] = azure_phrases_vis
df_yelp_vis.to_csv('visitor.csv')

In [47]:
df_yelp_vis

,review_date,address_zipcode,review_text,review_rating,price,NYC_reviewer,year,phrase
0,2015-10-13,11432.0,"Classic Urban Deli.. they have the ""hungry man...",5,$,0.0,2015,"[nice sandwich, breakfast sandwich sausage bac..."
1,2015-02-24,10033.0,"Meh it's a deli, they have soda and milk and y...",2,$,0.0,2015,"[nice Spanish bakery, Jin's, milk, block, weir..."
2,2014-05-13,10033.0,"This is the newest deli on the block, and I th...",4,$,0.0,2014,"[newest deli, newest addition, deli sandwiches..."
4,2015-05-12,11234.0,This is a rare gem in the area. Not too often ...,4,UNCLAIMED RESTAURANT,0.0,2015,"[fried seafood, fried fish, fried rice, crab m..."
6,2015-04-19,11234.0,An above average bar in an above average neigh...,4,$$,0.0,2015,"[average bar, order, average neighborhood, Ran..."
...,...,...,...,...,...,...,...,...
18636,2012-07-18,11234.0,"Last night, I was asked the question, ""If you ...",4,$$,0.0,2012,"[breakfast sandwiches, AvocadoLeast favorite s..."
18637,2013-10-17,11234.0,Clean location with friendly staff! The salads...,4,$$,0.0,2013,"[Clean location, friendly staff, generous port..."
18640,2012-10-24,11234.0,Um honestly glad this came to KP. There isn't ...,5,$$,0.0,2012,"[favorite friendly person, Aroushia, service, ..."
18642,2013-04-14,11234.0,Very nice service and the mango smoothie was c...,5,$$,0.0,2013,"[mango smoothie, nice service]"


In [54]:
dict_non_nyc =dict()
for i in azure_phrases_vis:
    for j in i:
        j = j.lower()
        dict_non_nyc[j] = dict_non_nyc.get(j,0) + 1
df_vis = pd.DataFrame.from_dict(dict_non_nyc, orient='index')
df_vis = df_vis.sort_values(by =0, ascending = False)
df_vis = df_vis.rename(columns ={0: 'count'})
df_vis.to_csv('vis_phrase.csv')

In [55]:
df_vis

,count
food,2511
place,2401
service,1577
time,1181
menu,758
...,...
china town,1
north dumpling,1
bianco slice,1
grandmas slice,1


In [50]:
word_vis =[]
for phrase in azure_phrases_vis:
    for i in phrase:
        l = i.split(' ')
        for j in l:
            j = j.lower()
            word_vis.append(j)

dict_word_vis =dict()
for i in word_vis:
    dict_word_vis[i] = dict_word_vis.get(i,0) + 1
df_word_vis = pd.DataFrame.from_dict(dict_word_vis, orient='index')
df_word_vis = df_word_vis.sort_values(by =0, ascending = False)
df_word_vis = df_word_vis.rename(columns ={0: 'count'})
df_word_vis.to_csv('vis_word.csv')

In [51]:
df_word_vis

,count
food,5335
of,4731
place,3758
service,2852
chicken,2491
...,...
lewisburg,1
remedy,1
hungriest,1
duarte,1


In [46]:
res =[]
for i in azure_phrases_res:
    for j in i:
        res.append(j)
res

['best shrimp',
 'fried baby shrimp',
 'broccoli',
 'garlic',
 'neighborhood',
 'seafood',
 'gym',
 'point',
 'restaurant',
 'dish',
 'Delivery',
 'best Chinese',
 'neighborhood',
 'food',
 'restaurant',
 'local people',
 'good local option',
 'quality',
 'friends',
 'bar',
 'neighborhood',
 'drinks',
 'cooks',
 'turn',
 'taste',
 'family',
 'Staff',
 'food',
 'game',
 'plus',
 'way',
 'house',
 'steak sandwich',
 'Nachos',
 'food',
 'Cozy',
 'Great service',
 'great food',
 'neighborhood bar',
 'minute walk',
 'Park Slope',
 'home',
 'afternoon',
 'soccer game',
 'Lovely decor',
 'mimosas',
 'dinner',
 'dessert',
 'plate',
 'sliders',
 'slider mini burgers',
 'mozzarella sticks',
 'zucchini sticks',
 'hot sauce',
 'sampler of poppers',
 'sampler platter',
 'honey BBQ wings',
 'room',
 'order',
 'size of meatballs',
 'appetizers',
 'months',
 'genius',
 'portion sizes',
 'boyfriend',
 'beers',
 'staff',
 'insult',
 'food',
 'rest',
 'menu',
 'pasta',
 'amazing - i',
 'great bar',
 'tim

# Question 4


# 4a - All plots in excel file 4a

In [37]:
#List of origins
zipcode_list=panel['zipcode'].unique()
zipcode_list=zipcode_list.astype('int')
zipcode_list=zipcode_list.astype('str')

#List of Destinations
destinations=['StatueofLibertyNationalMonument','CentralPark','TimesSquare','EmpireStateBuilding','RockerfellerCenter']

In [29]:
#Generating links for combinations of origin and destination
links=[]
origin=[]
destination=[]
for i in range(len(zipcode_list)):
    for k in range(len(destinations)):
        l='https://maps.googleapis.com/maps/api/distancematrix/json?origins='+destinations[k]+'&destinations='+zipcode_list[i]+'&units=imperial&key='
        links.append(l)
        destination.append(destinations[k])
        origin.append(zipcode_list[i])

In [30]:
### Getting distances between origin and destinations
resultdict=[]
distance=[]
yourKey = 'AIzaSyD8sXtVoP6pyz9C81z5_MK72LFoM6Y'
import urllib.request, urllib.error, urllib.parse
import json
for i in range(len(links)):
    results_unstructured = urllib.request.urlopen(links[i]+yourKey).read()
    results = json.loads(results_unstructured)
    dist=results.get('rows')[0].get('elements')[0].get('distance').get('text')
    resultdict.append(results)
    distance.append(dist)

In [31]:
#Creating a dataframe 
df=pd.DataFrame()
df['Origin']=origin
df['Destination']=destination
df['Distance']=distance

#extracting the airbnb activity from panel dataframe
airbnbactivity = panel.copy()
airbnbactivity['zipcode']=airbnbactivity['zipcode'].astype(int)
airbnbactivity['zipcode']=airbnbactivity['zipcode'].astype(str)

#aggregating on zipcode level to plot the graphs
airbnbactivity=airbnbactivity.groupby(['zipcode'], as_index=False).agg({'airbnb':np.sum})
plot=airbnbactivity.merge(df, left_on='zipcode', right_on='Origin')
plot.to_csv('plot.csv')

# 4b - All plots in excel file 4b and plot2_4b

In [ ]:
#List of Destinations
destinations=['Manhattan','Queens','Brooklyn','TheBronx','StatenIsland']

zipcode_list=panel['zipcode'].unique()
zipcode_list=zipcode_list.astype('int')
zipcode_list=zipcode_list.astype('str')

#Generating links for combinations of origin and destination
links=[]
origin=[]
destination=[]
for i in range(len(zipcode_list)):
    for k in range(len(destinations)):
        l='https://maps.googleapis.com/maps/api/distancematrix/json?origins='+destinations[k]+'&destinations='+zipcode_list[i]+'&units=imperial&key='
        links.append(l)
        destination.append(destinations[k])
        origin.append(zipcode_list[i])
        
        
### Getting distances between origin and destinations
resultdict=[]
distance=[]
yourKey = 'AIzaSyD8sXtVoP6pyz9C81z5_MK72LFoM6Y'
import urllib.request, urllib.error, urllib.parse
import json
for i in range(len(links)):
    results_unstructured = urllib.request.urlopen(links[i]+yourKey).read()
    results = json.loads(results_unstructured)
    dist=results.get('rows')[0].get('elements')[0].get('distance').get('text')
    resultdict.append(results)
    distance.append(dist)

In [ ]:
#Creating a dataframe 
df=pd.DataFrame()
df['Origin']=origin
df['Destination']=destination
df['Distance']=distance

#grouping the airbnb activity at zip code level
airbnbactivity = panel.copy()
airbnbactivity['zipcode']=airbnbactivity['zipcode'].astype(int)
airbnbactivity['zipcode']=airbnbactivity['zipcode'].astype(str)
airbnbactivity=airbnbactivity.groupby(['zipcode'], as_index=False).agg({'airbnb':np.sum})

#exporting to plot1 for further analysis
plot1=airbnbactivity.merge(df, left_on='zipcode', right_on='Origin')
plot1.to_csv('plot1.csv')

In [ ]:
#reading secondary data and filtering for zip codes
airbnb = pd.read_csv('listings.csv')
airbnb['zipcode'] = airbnb['zipcode'].str.replace('NY ', '')
airbnb_filter = airbnb[airbnb['zipcode'].isin(zipcode_list)]

#export to csv
airbnb_filter.to_csv('listings_filtered.csv')

In [ ]:
#grouping the data by zipcodes
airbnb1 = airbnb_filter.groupby(['zipcode']).agg({'price':'mean','reviews_per_month':'mean','review_scores_location': 'mean','number_of_reviews':'sum','availability_30':'mean','availability_60':'mean','availability_90':'mean','availability_365':'mean'}).reset_index()

plot2=airbnbactivity.merge(airbnb1, left_on='zipcode', right_on='zipcode')
plot2.to_csv('plot2.csv')